## Final Project Submission

Please fill out:
* Student name: Benjamin Bai, Nicolas Pierce, Alvaro Mendizabal
* Student pace: Full Time
* Scheduled project review date/time: 8/5/2022
* Instructor name: David Elliott
* Blog post URL: (TBD)


### Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import warnings
warnings.filterwarnings(action='ignore')
import folium
KC_coord = (47.4081,-121.9949)
Seattle_coord = (47.6062,-122.3321)
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-notebook')
from yellowbrick.regressor import ResidualsPlot
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OrdinalEncoder, OneHotEncoder

### DataFrame